<a href="https://www.kaggle.com/code/i200605salehahmad/100-f1-score-classification-adaboost?scriptVersionId=131128289" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install lazypredict

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

import lazypredict
from lazypredict.Supervised import LazyClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv('/kaggle/input/easiest-diabetes-classification-dataset/Diabetes Classification.csv')
df

,Age,Gender,BMI,Blood Pressure,FBS,HbA1c,Family History of Diabetes,Smoking,Diet,Exercise,Diagnosis
0,45,Male,25,Normal,100,5.70,No,No,Healthy,Regular,No
1,55,Female,30,High,120,6.40,Yes,Yes,Poor,No,Yes
2,65,Male,35,High,140,7.10,Yes,Yes,Poor,No,Yes
3,75,Female,40,High,160,7.80,Yes,Yes,Poor,No,Yes
4,40,Male,20,Normal,80,5.00,No,No,Healthy,Regular,No
...,...,...,...,...,...,...,...,...,...,...,...
123,17,Female,15,Normal,100,5.70,No,Yes,Poor,No,Yes
124,22,Male,19,Normal,120,6.40,No,Yes,Poor,No,Yes
125,27,Female,24,High,140,7.10,No,Yes,Poor,No,Yes
126,32,Male,29,High,160,7.80,No,Yes,Poor,No,Yes


In [4]:
X = df.drop('Diagnosis', axis=1)
Y = df['Diagnosis']

np.shape(X), np.shape(Y)

((128, 10), (128,))

In [5]:
#Check for nulls in the dataset
X.isnull().sum()

Age                           0
Gender                        0
BMI                           0
Blood Pressure                0
FBS                           0
HbA1c                         0
Family History of Diabetes    0
Smoking                       0
Diet                          0
Exercise                      0
dtype: int64

In [6]:
AllNumericalCols = [col for col in X.columns if df[col].dtypes!='O']
AllNumericalCols

['Age', 'BMI', 'FBS', 'HbA1c']

In [7]:
AllCategoricalCols = set(X.columns) - set(AllNumericalCols)
AllCategoricalCols

{'Blood Pressure',
 'Diet',
 'Exercise',
 'Family History of Diabetes',
 'Gender',
 'Smoking'}

In [8]:
#NOrmalize numerical columns
scaler = StandardScaler()
X[AllNumericalCols] = scaler.fit_transform(X[AllNumericalCols])
X

,Age,Gender,BMI,Blood Pressure,FBS,HbA1c,Family History of Diabetes,Smoking,Diet,Exercise
0,0.18,Male,-0.69,Normal,-1.02,-1.02,No,No,Healthy,Regular
1,0.78,Female,-0.36,High,-0.70,-0.70,Yes,Yes,Poor,No
2,1.37,Male,-0.02,High,-0.37,-0.37,Yes,Yes,Poor,No
3,1.97,Female,0.31,High,-0.04,-0.04,Yes,Yes,Poor,No
4,-0.12,Male,-1.03,Normal,-1.35,-1.35,No,No,Healthy,Regular
...,...,...,...,...,...,...,...,...,...,...
123,-1.50,Female,-1.36,Normal,-1.02,-1.02,No,Yes,Poor,No
124,-1.20,Male,-1.10,Normal,-0.70,-0.70,No,Yes,Poor,No
125,-0.90,Female,-0.76,High,-0.37,-0.37,No,Yes,Poor,No
126,-0.60,Male,-0.43,High,-0.04,-0.04,No,Yes,Poor,No


In [9]:
#Label Encode Categorical Columns
LE = LabelEncoder()
for idxcol,col in enumerate(AllCategoricalCols):
    X[col] = LE.fit_transform(X[col])
X

,Age,Gender,BMI,Blood Pressure,FBS,HbA1c,Family History of Diabetes,Smoking,Diet,Exercise
0,0.18,1,-0.69,2,-1.02,-1.02,0,0,0,1
1,0.78,0,-0.36,0,-0.70,-0.70,1,1,1,0
2,1.37,1,-0.02,0,-0.37,-0.37,1,1,1,0
3,1.97,0,0.31,0,-0.04,-0.04,1,1,1,0
4,-0.12,1,-1.03,2,-1.35,-1.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
123,-1.50,0,-1.36,2,-1.02,-1.02,0,1,1,0
124,-1.20,1,-1.10,2,-0.70,-0.70,0,1,1,0
125,-0.90,0,-0.76,0,-0.37,-0.37,0,1,1,0
126,-0.60,1,-0.43,0,-0.04,-0.04,0,1,1,0


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

np.shape(X_train),np.shape(X_test),np.shape(Y_train),np.shape(Y_test)

((96, 10), (32, 10), (96,), (32,))

In [11]:
#Call Lazy Predict
LZ = LazyClassifier()
models,predictions = LZ.fit(X_train, X_test, Y_train, Y_test)
models

100%|██████████| 29/29 [00:00<00:00, 30.61it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SGDClassifier,1.00,1.00,None,1.00,0.02
Perceptron,1.00,1.00,None,1.00,0.02
LinearSVC,1.00,1.00,None,1.00,0.01
AdaBoostClassifier,0.97,0.95,None,0.97,0.10
BaggingClassifier,0.97,0.95,None,0.97,0.03
SVC,0.97,0.95,None,0.97,0.01
RidgeClassifierCV,0.97,0.95,None,0.97,0.01
RidgeClassifier,0.97,0.95,None,0.97,0.02
RandomForestClassifier,0.97,0.95,None,0.97,0.15


# As from lazy predict top classifier is Adaboost so, calling ada boost on another set of train and test splits

In [12]:
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

model = AdaBoostClassifier()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

acc = accuracy_score(Y_test, Y_pred)*100
prec = precision_score(Y_test, Y_pred, average='macro')*100
rec = recall_score(Y_test, Y_pred, average='macro')*100
f1 = f1_score(Y_test, Y_pred, average='macro')*100

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  96.875
Precision:  97.72727272727273
Recall:  95.45454545454545
F1 Score:  96.45625692137321
